In [19]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [20]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [21]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [22]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [23]:
import bead_util as bu
import transfer_func_util as tf
import configuration as config

### Global parameters

In [24]:
bandwidth = 2
decimate = 10
drive_freq = 139
fsamp = 5000

### Discharge data

Fitting discharge data in y to extract the scale

In [25]:
ll = likelihood_analyser.LikelihoodAnalyser()

## extracing the amplitude at a certain frequency 
def func1_scale(i):
    
    bb = step_file_objs[i]      
    
    ## extract drive phase
    freq2 = np.fft.rfftfreq(len(bb.other_data[5]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    phase_diff = 0
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'fix_sigma': False, 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}
    
    ## bandpass data
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    ## extract amplitude
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], m1_tmp.values[2]-phi_tmp2

In [26]:
## load all data to a list - step_file_objs
step_cal_dir = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/'
max_file = 500

step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

175


In [27]:
## run the function on all calibration files
amps = []
for i in range(num_calib_files):
    print(i,'/',len(step_cal_files))
    amps.append(func1_scale(i))

0 / 175
1 / 175
2 / 175
3 / 175
4 / 175
5 / 175
6 / 175
7 / 175
8 / 175
9 / 175
10 / 175
11 / 175
12 / 175
13 / 175
14 / 175
15 / 175
16 / 175
17 / 175
18 / 175
19 / 175
20 / 175
21 / 175
22 / 175
23 / 175
24 / 175
25 / 175
26 / 175
27 / 175
28 / 175
29 / 175
30 / 175
31 / 175
32 / 175
33 / 175
34 / 175
35 / 175
36 / 175
37 / 175
38 / 175
39 / 175
40 / 175
41 / 175
42 / 175
43 / 175
44 / 175
45 / 175
46 / 175
47 / 175
48 / 175
49 / 175
50 / 175
51 / 175
52 / 175
53 / 175
54 / 175
55 / 175
56 / 175
57 / 175
58 / 175
59 / 175
60 / 175
61 / 175
62 / 175
63 / 175
64 / 175
65 / 175
66 / 175
67 / 175
68 / 175
69 / 175
70 / 175
71 / 175
72 / 175
73 / 175
74 / 175
75 / 175
76 / 175
77 / 175
78 / 175
79 / 175
80 / 175
81 / 175
82 / 175
83 / 175
84 / 175
85 / 175
86 / 175
87 / 175
88 / 175
89 / 175
90 / 175
91 / 175
92 / 175
93 / 175
94 / 175
95 / 175
96 / 175
97 / 175
98 / 175
99 / 175
100 / 175
101 / 175
102 / 175
103 / 175
104 / 175
105 / 175
106 / 175
107 / 175
108 / 175
109 / 175
110 / 175


In [28]:
amps = np.array(amps)
scaleY = np.abs(np.mean(amps[150:,0]))/4 ## scale is taken from the highest step

_,ax = plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(amps)), amps[:,0]/scaleY, label='fit phase')

ax.set(xlabel='dataset', ylabel='electron charges', ylim=(-0.3,4.5), xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
from matplotlib.ticker import MaxNLocator

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
print("Scale is ", scaleY)
bb = step_file_objs[-1]
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage_calib = np.std(xx2)*np.sqrt(2)*100
print("voltage_calib: ", voltage_calib)


Scale is  97.9396835176
voltage_calib:  24.0265126779


In [46]:
print("Scale is ", scaleY)

bb = step_file_objs[-1]
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage_calib = np.std(xx2)*np.sqrt(2)*100
print("voltage_calib: ", voltage_calib)


Scale is  97.9396835176
voltage_calib:  24.7304750766


### Single electrode data in Z and Y

In [30]:
## same procedure as above, but in here I extract the single electrode response at unknown charge state
ll = likelihood_analyser.LikelihoodAnalyser()

def get_plus_minus_Z(i):
    
    ## load file
    fname = r'/data/old_trap/20211001/bead1/single_electrode_20211013/Z_plus_139Hz_50Vpp/Z_50Vpp_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    ## extract drive phase
    freq2 = np.fft.rfftfreq(len(bb.other_data[1]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1]))

    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    
    phase_diff = 5 # initial guess for the response shift
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'fix_sigma': False, 'fix_phi': False, 'fix_f': True,
                  'print_level': 0}  
    
    
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    
    ## fit the signal
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    
    ## same for minus electrode
    fname = r'/data/old_trap/20211001/bead1/single_electrode_20211013/Z_minus_139Hz_50Vpp/Z_50Vpp_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    freq2 = np.fft.rfftfreq(len(bb.other_data[2]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[2]))

    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    phase_diff = 5 # initial guess
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'fix_sigma': False, 'fix_phi': False, 'fix_f': True,
                  'print_level': 0}  
    
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m1_tmp.values[0], m2_tmp.values[0]

In [31]:
plus_minus_Z = np.array([get_plus_minus_Z(i) for i in range(50)])

In [32]:
ll = likelihood_analyser.LikelihoodAnalyser()

def get_plus_minus_Y(i):
    
    ## load file
    fname = r'/data/old_trap/20211001/bead1/single_electrode_20211013/Y_plus_139Hz_50Vpp/Y_50Vpp_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    ## extract drive phase
    freq2 = np.fft.rfftfreq(len(bb.other_data[5]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5]))

    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    phase_diff = 0
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'fix_sigma': False, 'fix_phi': False, 'fix_f': True,
                  'print_level': 0}  
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    ## fit the signal
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    
    ## same for minus electrode
    fname = r'/data/old_trap/20211001/bead1/single_electrode_20211013/Y_minus_139Hz_50Vpp/Y_50Vpp_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))

    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    phase_diff = 0
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'fix_sigma': False, 'fix_phi': False, 'fix_f': True,
                  'print_level': 0}  
        
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m1_tmp.values[0], m2_tmp.values[0]

In [33]:
plus_minus_Y = np.array([get_plus_minus_Y(i) for i in range(50)])

In [34]:
response_elec5 = np.mean(plus_minus_Y[:,0])
response_elec6 = np.mean(plus_minus_Y[:,1])
print('electrodes response - y: ', response_elec5, response_elec6, response_elec5/response_elec6)

response_elec1 = np.mean(plus_minus_Z[:,0])
response_elec2 = np.mean(plus_minus_Z[:,1])
print('electrodes response - z: ', response_elec1, response_elec2, response_elec1/response_elec2)

electrodes response - y:  383.312679369 -396.604425653 -0.966486137258
electrodes response - z:  -408.513407393 611.114759232 -0.668472494277


In [35]:
## now we can extract the Z scale
scaleZ = scaleY*(np.abs(response_elec1)+np.abs(response_elec2))/(np.abs(response_elec5)+np.abs(response_elec6))

In [36]:
scaleY, scaleZ

(97.939683517624943, 128.04188971089127)

### COMSOL factor

In [37]:
fname = r"top_z_axis_efeild_paper.txt"
Ez_top = np.loadtxt(fname, comments='%')
fname = r"bottom_z_axis_efeild_paper.txt"
Ez_bottom = np.loadtxt(fname, comments='%')

In [38]:
_,ax = plt.subplots(1,2,figsize=(10,4))

ax[0].scatter(Ez_top[:,2]*1000, Ez_top[:,3], label='Top electrode')
ax[0].scatter(Ez_bottom[:,2]*1000, Ez_bottom[:,3], label='Bottom electrode')
ax[0].legend()
ax[0].set(xlabel=r'Z [mm]', ylabel=r'E$_Z$ [V/m]', xlim=(-1,1))

ax[1].plot(Ez_top[:,2]*1000,  Ez_top[:,3]/Ez_bottom[:,3], '.', label='Right/Left')
ax[1].set(xlabel=r'Z [mm]', ylabel=r'Ratio', xlim=(-1,1))

z_um = -0.2305 # guess the position
ratio = np.interp(z_um, Ez_top[:,2]*1000, Ez_top[:,3]/Ez_bottom[:,3])
field_plus = np.interp(z_um, Ez_top[:,2]*1000, Ez_top[:,3])
field_minus = np.interp(z_um, Ez_top[:,2]*1000, Ez_bottom[:,3])
print('Ratio is: ', ratio)
print('E-feilds are: ',field_plus, field_minus)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Ratio is:  -0.668671101898
E-feilds are:  -131.069382905 196.014798633


In [39]:
z_um1 = np.interp(z_um+0.01, Ez_top[:,2]*1000,  Ez_top[:,3])
z_um2 = np.interp(z_um-0.01, Ez_top[:,2]*1000,  Ez_top[:,3])

z_um3 = np.interp(z_um+0.01, Ez_bottom[:,2]*1000,  Ez_bottom[:,3])
z_um4 = np.interp(z_um-0.01, Ez_bottom[:,2]*1000,  Ez_bottom[:,3])

print('gradients ratio: ',(z_um1-z_um2)/(z_um3-z_um4))
print('gradients are [V/m2]: ', (z_um1-z_um2)/20*1e6, (z_um3-z_um4)/20*1e6)

gradients ratio:  0.618840520038
gradients are [V/m2]:  -108028.393151 -174565.804359
